Пользователь и все просмотренные им фильмы - user embedding, информация о фильме movie embedding.
Решим задачу регрессии, применяя loss MSE и таргет как функцию отклонения предполагаемой оценки от действительной. В результате получаем эмбеддинг содержащийся в модели обученной на просмотренных фильмах пользователя. 

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import sklearn as sk
import tensorflow as tf

In [2]:
#чтение таблиц
import yaml
with open("config.yaml","r") as y:
    cfg = yaml.safe_load(y)

movie = pd.read_csv(cfg["embeddings"]["movies"])
rating = pd.read_csv(cfg["embeddings"]["ratings"])
print(movie.info())
print(rating.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
None


Подготовка таблицы movies. Обработка категориальных переменных.

In [3]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

#OrdinalEncoder ~ Label с возможностью обрабатывать особые значения.
label_enc1 = OrdinalEncoder()
label_enc2 = LabelEncoder()
df = pd.DataFrame(movie["genres"].str.split("|").values.tolist()).fillna("None")
m = np.reshape(movie["genres"].str.split("|"),-1)

#составление выборки содержащей каждый tag
tr = sum(m, ["None"])
label_enc1.fit(np.reshape(tr,(-1,1)))

for i in df.columns:
    df[i] = label_enc1.transform(np.reshape(df[i],(-1,1)))
movie["title"] = label_enc2.fit_transform(movie["title"])

movie = movie.join(df)
movie = movie.drop("genres",axis=1)


rating - содержит данные о каждом пользователе по которым будет составлен эмбдеддинг. Так система будет индивидуально обучаться для каждого пользователя $\Rightarrow$ выше точность, также появляется возможность сравнивать объекты типа пользователь.

In [4]:
rating = rating.merge(movie, on="movieId")
rating = rating.drop("timestamp", axis=1)
rating

,userId,movieId,rating,title,0,1,2,3,4,5,6,7,8,9
0,1,1,4.0,8888,2.0,3.0,4.0,5.0,9.0,15.0,15.0,15.0,15.0,15.0
1,5,1,4.0,8888,2.0,3.0,4.0,5.0,9.0,15.0,15.0,15.0,15.0,15.0
2,7,1,4.5,8888,2.0,3.0,4.0,5.0,9.0,15.0,15.0,15.0,15.0,15.0
3,15,1,2.5,8888,2.0,3.0,4.0,5.0,9.0,15.0,15.0,15.0,15.0,15.0
4,17,1,4.5,8888,2.0,3.0,4.0,5.0,9.0,15.0,15.0,15.0,15.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,160341,2.5,1193,1.0,18.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0
100832,610,160527,4.5,8267,1.0,6.0,8.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0
100833,610,160836,3.0,3824,1.0,8.0,18.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0
100834,610,163937,3.5,1150,11.0,18.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0


In [5]:
# обучение своей модели для каждого пользователя индивидуально -- (малый объем данных для некоторых)?.
users = []
for i in rating["userId"].unique():
    users.append((rating.loc[rating["userId"] == i]).drop(["userId","movieId"],axis=1))

print(users[0])

       rating  title     0     1     2     3     4     5     6     7     8   
0         4.0   8888   2.0   3.0   4.0   5.0   9.0  15.0  15.0  15.0  15.0  \
215       4.0   3667   5.0  16.0  15.0  15.0  15.0  15.0  15.0  15.0  15.0   
267       4.0   3851   1.0   6.0  18.0  15.0  15.0  15.0  15.0  15.0  15.0   
369       5.0   7539  14.0  18.0  15.0  15.0  15.0  15.0  15.0  15.0  15.0   
572       5.0   9137   6.0  14.0  18.0  15.0  15.0  15.0  15.0  15.0  15.0   
...       ...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
16056     4.0   7574   1.0   6.0  18.0  15.0  15.0  15.0  15.0  15.0  15.0   
16075     5.0   9633   1.0   2.0  17.0  15.0  15.0  15.0  15.0  15.0  15.0   
16208     4.0   9370   5.0  15.0  15.0  15.0  15.0  15.0  15.0  15.0  15.0   
16243     4.0   8912   2.0   3.0   4.0  17.0  15.0  15.0  15.0  15.0  15.0   
16250     5.0   5315   5.0   8.0  19.0  15.0  15.0  15.0  15.0  15.0  15.0   

          9  
0      15.0  
215    15.0  
267    15.0  
369    

Подготовка train, test наборов данных для обучения модели.

In [6]:
from sklearn.model_selection import train_test_split

#выбор таргета
target = ["rating"]
features = list(users[0].columns)
features.remove("rating")

#обучающая,тестовая выборка для каждого пользователя
splits = []
for i in users:
    x = i[features]
    y = i[target]
    x_train, x_test, y_train, y_test = train_test_split(x.values,y.values.reshape(-1),train_size=0.8,shuffle=True)
    splits.append([x_train,x_test,y_train,y_test])


Обучение индивидуальной модели для каждого пользователя. Метрика выбрана MSE т.к является непрерывной дифференцируемой функцией, довольно простая для понимания, показывает хорошие результаты в других моделях.  
Архитектура сети представляет собой модель из 3х слоев, где начальный слой формирует эмбеддинг, а второй и третий позволяют системе предсказывать оценку. Из за небольшого количества слоев, обратное распространение ошибки больше влияет на слой эмбеддинга. 

In [7]:
models = []
for i in splits:
    m = tf.keras.Sequential()
    m.add(tf.keras.layers.Embedding(math.ceil(np.amax(rating.values)+1), 10, name="embedding"))
    m.add(tf.keras.layers.Dense(16, activation='relu'))
    m.add(tf.keras.layers.Dense(1))
    m.compile(loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
    m.fit(i[0],i[2],epochs=100, verbose=0 ,use_multiprocessing=True)
    ##полученные эмбеддинги
    #print(m.evaluate(i[1],i[3],verbose=0),m.get_layer("embedding").get_weights()[0])

В рамках реализованной модели, близость "к вкусам пользователя" можно оценить вызвав predict() по вектору признаков фильма. Чем ближе значение к 5.0 тем больше вероятность рекомендовать данный фильм. Или определить расстояние и провести оценку предсказанного эмбеддинга с пользовательскими. Аналогично можно ввести расстояние между пользователями. 

#todo: 
визуализация, оценка рекомендаций, определить расстояния, глобальные эмбеддинги.